In [ ]:
%run swipdg.ipynb

In [ ]:
import numpy as np

In [ ]:
print('estimating error ...')
print('')

from dune.xt.functions import make_esv2007_cutoff_function_diffusion_factor_and_tensor_to_1x1 as make_esv2007_cutoff_function

from dune.gdt import (
    #make_elliptic_matrix_operator_istl_row_major_sparse_matrix_double as make_elliptic_matrix_operator,
    #apply_oswald_interpolation_operator,
    make_rt_leaf_view_to_2x1_pdelab_p0_space as make_rt_space,
    #make_fv_leaf_view_to_1x1_gdt_space as make_fv_space,
    #project,
    apply_diffusive_flux_reconstruction_operator,
    apply_weighted_l2_product_leaf_view as apply_weighted_l2_product,
    make_local_diffusive_flux_estimation_esv2007_operator_to_1x1,
    ResultStorage
)

#elliptic_operator_mu_bar = make_elliptic_matrix_operator(lambda_, kappa, space)

# the nonconformity indicator
#u_h_in_H10_vector = Vector(space.size(), 0.0)
#u_h_in_H10 = make_discrete_function(space, u_h_in_H10_vector, 'u_h_in_H10')
#apply_oswald_interpolation_operator(u_h, u_h_in_H10)
#u_h_in_H10.visualize('../u_h_in_H10')
#make_discrete_function(space, u_h_vector - u_h_in_H10_vector, 'u_h - u_h_in_H10').visualize('../u_h__minus__u_h_in_H10')
#eta_nc = elliptic_operator_mu_bar.induced_norm(u_h_vector - u_h_in_H10_vector)
#print('  nonconformity indicator: {} (should be 1.66e-01)'.format(eta_nc))

# alternative computation:
from dune.gdt import make_ESV2007_nonconformity_product_leaf_part
nonconformity_product = make_ESV2007_nonconformity_product_leaf_part(grid, -1, -1, boundary_info,
                                                                     lambda_, kappa,
                                                                     u_h, u_h,
                                                                     over_integrate=2)
eta_nc = np.sqrt(nonconformity_product.apply2())
print('  nonconformity indicator: {} (should be 1.66e-01)'.format(eta_nc))

# the residual indicator
rt_space = make_rt_space(grid)
flux_reconxtruction_vector = Vector(rt_space.size(), 0.0)
flux_reconstruction = make_discrete_function(rt_space, flux_reconxtruction_vector, 'flux reconstruction')
apply_diffusive_flux_reconstruction_operator(lambda_, kappa, u_h, flux_reconstruction)
#flux_divergence = flux_reconstruction.divergence()
#cutoff_function = make_esv2007_cutoff_function(grid, lambda_, kappa, name='Cutoff function')
#eta_r = np.sqrt(apply_weighted_l2_product(cutoff_function, f - flux_divergence, f - flux_divergence, grid, level=-1))
#print('  residual indicator: {} (should be 7.23e-02)'.format(eta_r))

# alternative computation:
from dune.gdt import make_ESV2007_residual_product_leaf_view
residual_product = make_ESV2007_residual_product_leaf_view(grid, -1, -1,
                                                           lambda_, kappa, f,
                                                           u_h, u_h,
                                                           over_integrate=2)
eta_r = np.sqrt(residual_product.apply2())
print('  residual indicator: {} (should be 7.23e-02)'.format(eta_r))

# the diffusive flux estimator
#local_diffusive_flux_estimation_operator = make_local_diffusive_flux_estimation_esv2007_operator_to_1x1(lambda_,
#                                                                                                        kappa,
#                                                                                                        flux_reconstruction, over_integrate=2)
#eta_df = ResultStorage()
#system_assembler.append(local_diffusive_flux_estimation_operator, u_h, u_h, eta_df)
#system_assembler.assemble()
#eta_df = np.sqrt(eta_df.result)
#print('  diffusive flux indicator: {} (should be 3.55e-01)'.format(eta_df))

# alternative computation:
from dune.gdt import make_ESV2007_diffusive_flux_product_leaf_view
diffusive_flux_product = make_ESV2007_diffusive_flux_product_leaf_view(grid, -1, -1,
                                                                       lambda_, kappa,
                                                                       u_h, u_h, over_integrate=2)
eta_df = np.sqrt(diffusive_flux_product.apply2())
print('  diffusive flux indicator: {} (should be 3.55e-01)'.format(eta_df))